<a href="https://colab.research.google.com/github/nagasivaninandam/Chatbot-LLM-Frontend-using-Streamlit/blob/master/Chatbot_LLM_Frontend_using_Streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Colab: install Python deps
!pip -q install streamlit openai tiktoken

# Colab: install Node LocalTunnel (no account needed)
!npm -g install localtunnel


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 84.7 MB/s eta 0:00:00
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋
added 22 packages in 2s
⠋
⠋3 packages are looking for funding
⠋  run `npm fund` for details
⠋

In [2]:
%%writefile app.py
import os
import time
from typing import List, Dict, Any

import streamlit as st

# --- Page setup ---
st.set_page_config(page_title="Chatbot / LLM Frontend (Streamlit)", page_icon="💬", layout="wide")

# --- Sidebar controls ---
st.sidebar.title("⚙️ Settings")

provider = st.sidebar.selectbox(
    "Model Provider",
    ["OpenAI", "Dummy (no key required)"],
    index=0
)

openai_key = st.sidebar.text_input("OpenAI API Key", type="password", help="Stored in memory for this session only.")
model_name = st.sidebar.text_input("Model name", value="gpt-4o-mini", help="Any chat-capable OpenAI model.")

temperature = st.sidebar.slider("Temperature", 0.0, 2.0, 0.7, 0.1)
max_tokens = st.sidebar.number_input("Max tokens (response)", min_value=64, max_value=4096, value=512, step=64)
system_prompt = st.sidebar.text_area(
    "System prompt",
    value="You are a helpful, concise assistant.",
    help="Sets the assistant's behavior/tone.",
    height=100
)

upload = st.sidebar.file_uploader("📎 Optional: upload a small .txt file for context", type=["txt"])
uploaded_context = ""
if upload is not None:
    try:
        uploaded_context = upload.read().decode("utf-8", errors="ignore")
        st.sidebar.success("Loaded context from file.")
    except Exception as e:
        st.sidebar.error(f"Couldn't read file: {e}")

st.sidebar.markdown("---")
st.sidebar.caption("Tip: Switch to **Dummy** to test the UI without any API key.")


# --- Session state for messages ---
if "messages" not in st.session_state:
    st.session_state.messages: List[Dict[str, Any]] = [
        {"role": "system", "content": system_prompt}
    ]

# Update system message live if user edits system_prompt
if len(st.session_state.messages) > 0 and st.session_state.messages[0]["role"] == "system":
    st.session_state.messages[0]["content"] = system_prompt

# Optional context injection
if uploaded_context:
    # Ensure there's a single context message right after system
    found_idx = None
    for i, m in enumerate(st.session_state.messages):
        if m.get("role") == "system" and i + 1 < len(st.session_state.messages) and st.session_state.messages[i+1].get("role") == "system" and m != st.session_state.messages[i+1]:
            # not used, but just being careful
            pass
    # Replace or insert a single "system" context message at index 1
    if len(st.session_state.messages) > 1 and st.session_state.messages[1].get("meta") == "context":
        st.session_state.messages[1]["content"] = f"Extra context from user file:\n\n{uploaded_context}"
    else:
        st.session_state.messages.insert(1, {
            "role": "system",
            "content": f"Extra context from user file:\n\n{uploaded_context}",
            "meta": "context"
        })


# --- Header ---
st.title("💬 Streamlit Chatbot / LLM Frontend")
st.caption("Supports OpenAI or a built-in Dummy mode. Try uploading a .txt file as context in the sidebar.")


# --- Chat history UI ---
for m in st.session_state.messages:
    if m["role"] == "system":
        # Don't render the system prompt in the main feed
        continue
    with st.chat_message("user" if m["role"] == "user" else "assistant"):
        st.markdown(m["content"])


# --- Helper: call OpenAI with v1 or legacy fallback ---
def call_openai_chat(messages: List[Dict[str, str]], model: str, temperature: float, max_tokens: int, api_key: str) -> str:
    """
    Tries OpenAI Python SDK v1 (from openai import OpenAI) first;
    falls back to legacy openai.ChatCompletion if needed.
    """
    os.environ["OPENAI_API_KEY"] = api_key

    # Try v1 client
    try:
        from openai import OpenAI
        client = OpenAI(api_key=api_key)
        # Convert messages to OpenAI format (they already are)
        resp = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=temperature,
            max_tokens=max_tokens,
        )
        return resp.choices[0].message.content
    except Exception as v1_err:
        # Try legacy fallback
        try:
            import openai
            openai.api_key = api_key
            resp = openai.ChatCompletion.create(
                model=model,
                messages=messages,
                temperature=temperature,
                max_tokens=max_tokens,
            )
            return resp["choices"][0]["message"]["content"]
        except Exception as legacy_err:
            raise RuntimeError(f"OpenAI call failed. v1 error: {v1_err}\nLegacy error: {legacy_err}")


# --- Helper: dummy responder (no API needed) ---
def dummy_response(user_text: str) -> str:
    # Silly but helpful for UI testing
    if not user_text.strip():
        return "Say something and I’ll reply! 🙂"
    return (
        "🤖 **Dummy mode** (no API):\n\n"
        f"• You said: `{user_text}`\n"
        f"• Reversed: `{user_text[::-1]}`\n"
        "• Tip: Switch provider to **OpenAI** and add your key in the sidebar to get real model responses."
    )


# --- Chat input ---
user_input = st.chat_input("Type your message")
if user_input is not None:
    # Add user message
    st.session_state.messages.append({"role": "user", "content": user_input})
    with st.chat_message("user"):
        st.markdown(user_input)

    # Generate assistant reply
    with st.chat_message("assistant"):
        with st.spinner("Thinking..."):
            try:
                if provider == "OpenAI":
                    if not openai_key:
                        raise ValueError("No OpenAI API key provided in the sidebar.")
                    # Prepare messages (ensure system prompt is at start)
                    msgs = [{"role": m["role"], "content": m["content"]} for m in st.session_state.messages]
                    reply = call_openai_chat(
                        messages=msgs,
                        model=model_name,
                        temperature=temperature,
                        max_tokens=max_tokens,
                        api_key=openai_key,
                    )
                else:
                    reply = dummy_response(user_input)
            except Exception as e:
                reply = f"⚠️ Error: {e}"

        st.markdown(reply)

    # Save assistant reply
    st.session_state.messages.append({"role": "assistant", "content": reply})


# --- Footer ---
st.markdown("---")
st.caption("Built with Streamlit. Dummy mode lets you test the interface without any API.")


Writing app.py


In [3]:
# Start Streamlit (background) and then expose port 8501 with LocalTunnel
# The LT command prints a public URL (copy it into your browser)
!streamlit run app.py & npx localtunnel --port 8501


⠙⠹⠸⠼

⠴⠦⠧⠇⠏⠋
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.192.43.223:8501

⠙⠹⠸your url is: https://spicy-frogs-tickle.loca.lt
  Stopping...
^C


In [ ]:
# --- Clean up anything still running ---
!pkill -f streamlit || true
!pkill -f node || true
!pkill -f localtunnel || true
!pkill -f cloudflared || true

# --- Start Streamlit in the background ---
!streamlit run app.py --server.enableCORS=false --server.enableXsrfProtection=false &>/tmp/app.log &

# --- Install & run Cloudflared (no password, public URL printed below) ---
!wget -q -O /usr/local/bin/cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x /usr/local/bin/cloudflared
!cloudflared tunnel --url http://localhost:8501 --no-autoupdate


In [ ]:
# --- Start Streamlit in the background ---
!streamlit run app.py --server.enableCORS=false --server.enableXsrfProtection=false &>/tmp/app.log &

# --- Install & run Cloudflared (no password, public URL printed below) ---
!wget -q -O /usr/local/bin/cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x /usr/local/bin/cloudflared
!cloudflared tunnel --url http://localhost:8501 --no-autoupdate

2025-10-19T08:31:22Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-10-19T08:31:22Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-10-19T08:31:25Z INF +--------------------------------------------------------------------------------------------+
2025-10-19T08:31:25Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-10-19T08:31:25Z INF |  https://usual-baseball-dive-appreciated.trycloudflare